In [1]:
import tensorflow as tf
import functools as ft
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from datetime import datetime as dt

In [46]:
n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_output = 10

learning_rate = 0.01
lambda_ = 0.
n_epochs = 25
batch_size = 100
tf.reset_default_graph()

now = dt.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "./CH10_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [47]:
X = tf.placeholder(tf.float32, shape = (None, n_inputs), name = "X")
y = tf.placeholder(tf.int64, shape = (None), name = "y")
is_training = tf.placeholder(tf.bool, shape = (), name = "is_training")

In [48]:
def neuron_layer(X, n_neurons, name, activation = None, weights_regularizer = None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev = stddev)
        W = tf.get_variable("{}_weights".format(name), initializer = init, regularizer = weights_regularizer)
        b = tf.Variable(tf.zeros([n_neurons]), name = "biasses")
        z = tf.matmul(X, W) + b
        if activation == "relu":
            return tf.nn.relu(z)
        elif activation == "elu":
            return tf.nn.elu(z)
        else:
            return z

def max_norm_regularizer(threshold, axes = 1, name = "max_norm", collection = "max_norm"):
    def max_norm(weights):
        clipped = tf.clip_by_norm(weights, clip_norm = threshold, axes = axes)
        clip_weights = tf.assign(weights, clipped, name = name)
        tf.add_to_collection(collection, clip_weights)
        return None
    return max_norm

In [49]:
my_dense_layer = ft.partial(neuron_layer, weights_regularizer = max_norm_regularizer(threshold = 1.0))

#my_dense_layer = ft.partial(neuron_layer, weights_regularizer = tf.contrib.layers.l2_regularizer(scale = lambda_))

with tf.name_scope("dnn"): 
    #normal1 = tf.layers.batch_normalization(X,training = is_training)
    #hidden1 = my_dense_layer(normal1, n_neurons = n_hidden1, name = "hidden1", activation = "elu")
    #normal2 = tf.layers.batch_normalization(hidden1, training = is_training)
    #hidden2 = my_dense_layer(normal2, n_neurons = n_hidden2, name = "hidden2", activation = "elu")
    #normal3 = tf.layers.batch_normalization(hidden2, training = is_training)
    #logits = my_dense_layer(normal3, n_neurons = n_output, name = "outputs")
    hidden1 = my_dense_layer(X, n_neurons = n_hidden1, name = "hidden1", activation = "elu")
    hidden2 = my_dense_layer(hidden1, n_neurons = n_hidden2, name = "hidden2", activation = "elu")
    logits = my_dense_layer(hidden2, n_neurons = n_output, name = "outputs")

with tf.name_scope("base_loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits)
    base_loss = tf.reduce_mean(xentropy, name = "base_loss")
'''    
with tf.name_scope("regularization"):
    reg_loss = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
'''
with tf.name_scope("loss"):
    #loss = tf.add_n([base_loss] + reg_loss)
    loss = base_loss

with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

clip_all_weights = tf.get_collection("max_norm")
init = tf.global_variables_initializer()
saver = tf.train.Saver()

acc_train_summary = tf.summary.scalar('Train_accuracy', accuracy)
acc_test_summary = tf.summary.scalar('Test_accuracy', accuracy)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [6]:
mnist = input_data.read_data_sets("/tmp/data/")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [50]:
with tf.Session()as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict = {X: X_batch, y: y_batch, is_training: True})
            sess.run(clip_all_weights)
            if iteration % 10 == 0 :
                acc_train_str = acc_train_summary.eval(feed_dict = {X : X_batch, y : y_batch, is_training: False})
                acc_test_str = acc_test_summary.eval(feed_dict = {X : mnist.test.images, y : mnist.test.labels, is_training: False})
                step = epoch * (mnist.train.num_examples // batch_size) + iteration
                file_writer.add_summary(acc_train_str, step)
                file_writer.add_summary(acc_test_str, step)
        #acc_train = accuracy.eval(feed_dict = {X: X_batch, y : y_batch, is_training: False})
        #acc_test = accuracy.eval(feed_dict = {X: mnist.test.images, y : mnist.test.labels, is_training: False})
        #print(epoch, " Train accuracy ", acc_train, " Test_accuracy ", acc_test)
    save_path = saver.save(sess,"./CH10/my_model_final.ckpt")
file_writer.close()